# Determinar los descriptores más relevantes
<hr>

In [1]:
import pandas as pd

In [2]:
des = pd.read_csv("df_con_descriptores_tratado.csv")

In [3]:
des.head()

,SMILES,BBB+/BBB-,mol,MaxAbsEStateIndex,MinEStateIndex,SPS,MolWt,FpDensityMorgan3,BCUT2D_MWHI,AvgIpc,...,fr_aryl_methyl,fr_bicyclic,fr_ester,fr_ether,fr_ketone,fr_methoxy,fr_para_hydroxylation,fr_piperdine,fr_sulfide,fr_unbrch_alkane
0,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,BBB-,<rdkit.Chem.rdchem.Mol object at 0x000001D68BE...,12.341010,-3.794932,11.428571,398.400,2.285714,32.233272,2.676891,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,BBB-,<rdkit.Chem.rdchem.Mol object at 0x000001D68B7...,13.190522,-2.144257,22.000000,520.480,2.555556,32.166556,3.253857,...,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0
2,Oc1c(I)cc(Cl)c2cccnc12,BBB-,<rdkit.Chem.rdchem.Mol object at 0x000001D68B7...,9.654043,0.195000,10.615385,305.502,3.000000,126.912704,2.146240,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CCNC(=NCCSCc1ncccc1Br)NC#N,BBB-,<rdkit.Chem.rdchem.Mol object at 0x000001D68B7...,8.544584,0.532052,10.894737,342.266,3.157895,79.918731,2.405844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,BBB-,<rdkit.Chem.rdchem.Mol object at 0x000001D68B7...,11.445328,-1.798901,45.303030,461.467,2.696970,16.712087,3.141364,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel